### Process API data


In [ ]:
from ingreedypy import Ingreedy
# Ingreedy is a recipe parser downloaded from GitHub
import pandas as pd

# Open json file with API data
import json
with open('allrecipes.json', 'r') as f:
     data = json.load(f)

print(len(data))


In [ ]:
# Drop everything that is not from allrecipes 

allrecipes = []
for i in range (1,len(data)):
    if data[i]['recipe']['publisher']=='All Recipes':
        allrecipes.append(data[i])
data=allrecipes
print len(data)

In [ ]:
# Drop duplicate recipes

data=[i for n, i in enumerate(data) if i not in data[n + 1:]]

### Create lookup lists for sugar, flour and fat, parse each recipe/ingredient, assign amts to new columns

In [ ]:
sugar_lkp=['sugar','honey','maple syrup']
fat_lkp=[' oil','butter','margarine','shortening']
flour_lkp=['flour','oatmeal','meal','cornmeal','oats']

# Conversion dictionary for recipe units
convert=dict([("cup",1),("fluid_ounce",0.13),("gallon",16),("ounce",0.13),("pint",2),("pound",2.25),("quart",4),
              ("tablespoon",0.06),("teaspoon",0.02),("gram",0.01),("kilogram",8),("liter",4.23)])

In [ ]:
def lkp(string1,list2):
    for word in list2:
        if word in string1:
            return True
    return False

In [ ]:
def sugar_fat(ingredients):
    sugar=[]
    fat=[]
    flour=[]
    amt_sugar=0
    amt_fat=0
    amt_flour=0
    #ingredients=data[i]['recipe']['ingredients']
    for item in ingredients:
        item=item.replace(',','')
        item=item.replace(')','')
        item=item.replace('(','')
        item=item.replace('-',' ')
        item=item.replace('&frac12','1/2')
        item=item.replace('&frac14','1/4')
        item=item.replace('&frac34','3/4')
        item=item.replace(';','')
        if lkp(item.lower(),sugar_lkp):
            try:
                p= Ingreedy().parse(item)
                p['amount']=convert[p['unit']]*p['amount']
                sugar.append(p)
            except:
                pass
        if lkp(item.lower(),fat_lkp):
            try:
                p= Ingreedy().parse(item)
                p['amount']=convert[p['unit']]*p['amount']
                fat.append(p)
            except:
                pass
        if lkp(item.lower(),flour_lkp):
            try:
                p= Ingreedy().parse(item)
                p['amount']=convert[p['unit']]*p['amount']
                flour.append(p)
            except:
                pass
    for i in range(0,len(sugar)):
        try:
            amt_sugar=amt_sugar+sugar[i]['amount']
        except:
            pass
    for i in range(0,len(fat)):
        try:
            amt_fat=amt_fat+fat[i]['amount']
        except:
            pass
    for i in range(0,len(flour)):
        try:
            amt_flour=amt_flour+flour[i]['amount']
        except:
            pass

    return amt_sugar, amt_fat, amt_flour


In [ ]:
# loop through each recipe and calculate amounts of fat, flour, sugar. Assign amts to new columns

for i in range(0,len(data)):
    recipe = data[i]['recipe']['ingredients']
    amt_sugar, amt_fat, amt_flour = sugar_fat(recipe)
    data[i]['recipe']['sugar']=amt_sugar
    data[i]['recipe']['fat']=amt_fat
    data[i]['recipe']['dry']=amt_flour

In [ ]:
# Drop recipe if it doesn't contain sugar

for i in range (0,len(data)-1):
    if data[i]['recipe']['sugar']==0:
        del data[i]
print len(data)

In [ ]:
# The data takes the form of a list of dictionaries
# Write the list of dictionaries to a csv file

keys=['social_rank','publisher','f2f_url','ingredients','source_url','recipe_id','image_url','publisher_url','title',
     'sugar','fat','dry']
import csv

with open('all.csv', 'wb') as f:  
    w = csv.DictWriter(f, keys, restval='')
    w.writeheader()
    for item in data:
        try:
            my_dict=item['recipe']
            w.writerow(my_dict)
        except:
            pass

In [ ]:
# Create a dataframe
# Add ratio columns

df=pd.read_csv('/Users/ranaquadri/Documents/recipe/master1.csv')

# Add ratio columns
df['ratio_sugar_flour']=df['sugar']/df['dry']
df['ratio_fat_flour']=df['fat']/df['dry']

#print df.head()
print df.info()

In [ ]:
  ## Create binary variables for positive adj and health adj in title

In [ ]:
lookup1=['addictive', 'affair', 'amazing', 'amazingly', 'awesome', 'best', 'bodacious', 'bomb', 'bombshell', 'bursting'
         , 'classic', 'decadence', 'decadent', 'delicious', 'die', 'easy', 'ever', 'excellent', 'fabulous', 'feel-good',
         'heavenly', 'irresistible', 'luscious', 'perfect', 'ravishing', 'restaurant-quality', 'righteous', 'sex', 
         'simple', 'ultimate', 'ultimate', 'upscale', 'very', 'yummy', 'world-famous']

lookup2=['calorie', 'fat-free', 'health', 'healthier', 'healthy', 'healthy-ish', 'gluten-free', 'lactose', 'light', 
         'lighter', 'low-carb', 'low-fat', 'makeover:', 'no-sugar-added', 'paleo', 'skinny', 'vegan', 'whole-grain']

In [ ]:
# Create binary column indicating whether positive adjectives are in the title
import regex as re
df['pos_adj']=0
for i in range(0,len(lookup1)):
    tt = lookup1[i] 
    for j in range(0,len(df)):
        if re.search(tt,df['title'][j].lower()):
            df['pos_adj'][j]=1                    
#print df['pos_adj'][0:5]

In [ ]:
# Create binary column indicating whether health adjectives are in the title

df['health_adj']=0
for i in range(0,len(lookup2)):
    tt = lookup2[i]   
    for j in range(0,len(df)):
        if re.search(tt,df['title'][j].lower()):
            df['health_adj'][j]=1
#print df['pos_adj'][0:5]

In [ ]:
df['text']=df["ingredients"].map(str) + df["title"].map(str)

In [ ]:
df['text'] = df['text'].str.replace('\d+', '')

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
no_features = 1000

# NMF is able to use tf-idf
tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=2, max_features=no_features, stop_words='english')
tfidf = tfidf_vectorizer.fit_transform(df['text'])
tfidf_feature_names = tfidf_vectorizer.get_feature_names()

# LDA can only use raw term counts for LDA because it is a probabilistic graphical model
tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2, max_features=no_features, stop_words='english')
tf = tf_vectorizer.fit_transform(df['text'])
tf_feature_names = tf_vectorizer.get_feature_names()

In [ ]:
from sklearn.decomposition import NMF, LatentDirichletAllocation
n_components = 10

# Run NMF
nmf = NMF(n_components=n_components, random_state=1, alpha=.1, l1_ratio=.5, init='nndsvd').fit(tfidf)
# Run LDA
lda = LatentDirichletAllocation(n_components=n_components, max_iter=5, learning_method='online', learning_offset=50.,random_state=0).fit(tf)

In [ ]:
def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print "Topic %d:" % (topic_idx)
        print " ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]])

no_top_words = 10
display_topics(nmf, tfidf_feature_names, no_top_words)
display_topics(lda, tf_feature_names, no_top_words)

In [ ]:
def display_topics(H, W, feature_names, df, no_top_words, no_top_documents):
    for topic_idx, topic in enumerate(H):
        print "Topic %d:" % (topic_idx)
        print " ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]])
        top_doc_indices = np.argsort( W[:,topic_idx] )[::-1][0:no_top_documents]
        for doc_index in top_doc_indices:
            print df[doc_index]

In [ ]:
ingreds = df['text'].fillna

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()

In [ ]:
# Use `fit` to learn the vocabulary of the titles
vectorizer.fit(ingreds)

In [ ]:

from sklearn.feature_extraction.text import CountVectorizer
# list of text documents
text = ingreds
# create the transform
vectorizer = CountVectorizer()
# tokenize and build vocab
vectorizer.fit(text)
# summarize
print(vectorizer.vocabulary_)
# encode document
vector = vectorizer.transform(text)
# summarize encoded vector
print(vector.shape)
print(type(vector))
print(vector.toarray())